In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests # library to handle requests

# import k-means from clustering stage
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

try:
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
except:
    !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
    from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

try:
    import folium # map rendering library
except:
    !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
    import folium # map rendering library
    
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

# Import Toronto Data

In [43]:
#read the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_url = requests.get(url).text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})

#Clean and Structure Data
matrix=[]
for tr in table.find_all("tr"):
    td=tr.find_all('td')
    matrix.append([x.text.strip() for x in td])
    

columns = ['PostalCode', 'Borough',  'Neighborhood']
canada_data = pd.DataFrame(matrix, columns = columns)
canada_data = canada_data[1:]
canada_data = canada_data.loc[canada_data[columns[1]] != "Not assigned"]
canada_data.loc[canada_data[columns[2]] == "Not assigned",columns[2]] = canada_data.loc[canada_data[columns[2]]=="Not assigned",columns[1]]
canada_data = canada_data.groupby([columns[0],columns[1]])[columns[2]].apply(lambda tags: ','.join(tags)).to_frame().reset_index()

latlng = pd.read_csv('https://cocl.us/Geospatial_data')
canada_merged = canada_data.join(latlng.set_index('Postal Code'), on='PostalCode')

print('Data downloaded!')
print("There are {} Boroughs and {} Neightborhoods in Canada".format(len(canada_merged['Borough'].unique()), canada_merged.shape[0]))

toronto_data = canada_merged[canada_merged['Borough'].str.contains("Toronto")]
toronto_data.reset_index(drop=True, inplace = True)
print("There are {} Boroughs and {} Neightborhoods in Toronto".format(len(toronto_data['Borough'].unique()),toronto_data.shape[0]))
                                                                                                                         
address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
toronto_location = geolocator.geocode(address)
toronto_latitude = toronto_location.latitude
toronto_longitude = toronto_location.longitude
print("The geograpical coordinate of Toronto are {}, {}.".format(toronto_latitude, toronto_longitude))

Data downloaded!
There are 11 Boroughs and 103 Neightborhoods in Canada
There are 4 Boroughs and 38 Neightborhoods in Toronto
The geograpical coordinate of Toronto are 43.653963, -79.387207.


# Import Paris Data

In [44]:
url = "https://opendata.paris.fr/explore/dataset/arrondissements/download"
url_pd = pd.read_csv(url, sep = ";")
print('Data downloaded!')

paris_data = url_pd['l_aroff']
loc = url_pd['geom_x_y'].str.split(",", n = 1, expand = True).astype(float)
paris_data = pd.concat([paris_data, loc], axis = 1)

column_names = ['Neighborhood', 'Latitude', 'Longitude'] 
paris_data.columns = column_names
paris_data
print('There are {} neighborhoods in Paris .'.format(paris_data.shape[0]))

address = 'Paris, Fr'

geolocator = Nominatim(user_agent="paris_explorer")
pr_location = geolocator.geocode(address)
pr_latitude = pr_location.latitude
pr_longitude = pr_location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(pr_latitude, pr_longitude))


Data downloaded!
There are 20 neighborhoods in Paris .
The geograpical coordinate of Paris are 48.8566101, 2.3514992.


# Import New York Data

In [45]:

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough, 'Neighborhood': neighborhood_name, 'Latitude': neighborhood_lat, 'Longitude': neighborhood_lon}, ignore_index=True)
    
print('There are {} boroughs and {} neighborhoods in New York.'.format(len(neighborhoods['Borough'].unique()), neighborhoods.shape[0]))
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
print('There are {} neighborhoods in Manhattan.'.format(manhattan_data.shape[0]))

address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
NY_location = geolocator.geocode(address)
NY_latitude = NY_location.latitude
NY_longitude = NY_location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(NY_latitude, NY_longitude))

address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
NY_location = geolocator.geocode(address)
NY_latitude = NY_location.latitude
NY_longitude = NY_location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(NY_latitude, NY_longitude))

Data downloaded!
There are 5 boroughs and 306 neighborhoods in New York.
There are 40 neighborhoods in Manhattan.
The geograpical coordinate of New York City are 40.7900869, -73.9598295.
The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=12)
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='red', fill=True, fill_color='r', fill_opacity=0.7,parse_html=False).add_to(map_toronto)
map_toronto
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[NY_latitude, NY_longitude], zoom_start=12)
# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='b', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_newyork)  
map_newyork

In [7]:
CLIENT_ID = 'HMHJ1U5UMMDL2JXQ0JHBAIIGJB1JHKLNIL5DWGMQY5FGNPOI' # your Foursquare ID
CLIENT_SECRET = 'QIJXKFG0WTN3A1BI3PULLIKUXRK0RC3OD51PTJVYBCZIXD2T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Your credentails:
CLIENT_ID: HMHJ1U5UMMDL2JXQ0JHBAIIGJB1JHKLNIL5DWGMQY5FGNPOI
CLIENT_SECRET:QIJXKFG0WTN3A1BI3PULLIKUXRK0RC3OD51PTJVYBCZIXD2T


In [46]:
try:
    toronto_data.insert(0,"City","Toronto")
    manhattan_data.insert(0,"City","Manhattan")
    paris_data.insert(0,"City","Paris")
except:
    next
merged = pd.DataFrame(toronto_data[["City", "Neighborhood", "Latitude", "Longitude"]].append(manhattan_data[["City", "Neighborhood", "Latitude", "Longitude"]]).append(paris_data[["City", "Neighborhood", "Latitude", "Longitude"]])).reset_index(drop=True)


In [47]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
paris_venues = getNearbyVenues(names=paris_data['Neighborhood'],
                                   latitudes=paris_data['Latitude'],
                                   longitudes=paris_data['Longitude']
                                  )

In [88]:
venues_merged = toronto_venues.append(manhattan_venues).append(paris_venues)
venues_merged.loc[venues_merged["Venue Category"].str.contains("Gym"),"Venue Category"] = "Gym"
venues_merged.loc[venues_merged["Venue Category"].str.contains("Café"), "Venue Category"] = "Coffee Shop"
venues_merged.loc[venues_merged["Venue Category"].str.contains("Coffee") ,"Venue Category"] = "Coffee Shop"

venues_merged["Venue Category"].value_counts()[0:20]

Coffee Shop            515
French Restaurant      248
Italian Restaurant     236
Hotel                  211
Gym                    167
Bakery                 164
Pizza Place            129
Bar                    127
Park                   110
Japanese Restaurant    109
American Restaurant    108
Cocktail Bar            94
Restaurant              92
Sandwich Place          81
Sushi Restaurant        80
Chinese Restaurant      76
Wine Bar                74
Ice Cream Shop          73
Mexican Restaurant      72
Seafood Restaurant      71
Name: Venue Category, dtype: int64

In [48]:
# one hot encoding
onehot = pd.get_dummies(venues_merged[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot.drop('Neighborhood', 1, inplace = True)
onehot.insert(0,'Neighborhood', venues_merged['Neighborhood'])

grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Basque Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burgundian Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Ch'ti Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / 

In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']


for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.insert(0, "City", merged["City"])

neighborhoods_venues_sorted.head()

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Toronto,"Adelaide,King,Richmond",Coffee Shop,Steakhouse,Bar,Thai Restaurant,American Restaurant,Gym,Cosmetics Shop,Hotel,Bakery,Burger Joint
1,Toronto,Batignolles-Monceau,Hotel,French Restaurant,Italian Restaurant,Bakery,Coffee Shop,Japanese Restaurant,Plaza,Bistro,Restaurant,Chinese Restaurant
2,Toronto,Battery Park City,Park,Coffee Shop,Hotel,Memorial Site,Wine Shop,Italian Restaurant,Clothing Store,Gym,Food Court,Men's Store
3,Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Seafood Restaurant,Italian Restaurant,Steakhouse,Cheese Shop,Beer Bar,Basketball Stadium
4,Toronto,Bourse,French Restaurant,Bistro,Wine Bar,Cocktail Bar,Salad Place,Japanese Restaurant,Bakery,Plaza,Hotel,Italian Restaurant


In [50]:
# set number of clusters
kclusters = 6

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_


array([1, 0, 5, 1, 0, 1, 5, 0, 0, 5, 1, 5, 1, 5, 5, 5, 1, 1, 1, 5, 5, 1, 1,
       5, 1, 1, 5, 5, 5, 0, 5, 1, 5, 4, 5, 5, 5, 1, 1, 1, 1, 5, 1, 0, 5, 4,
       5, 5, 5, 5, 0, 1, 0, 5, 5, 5, 5, 5, 3, 1, 5, 0, 5, 5, 0, 0, 0, 0, 5,
       0, 0, 4, 5, 4, 2, 1, 1, 5, 1, 1, 1, 5, 5, 5, 1, 5, 5, 1, 5, 5, 5, 5,
       5, 0, 5, 5, 5, 0], dtype=int32)

In [54]:
# add clustering labels
try:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
except:
    neighborhoods_venues_sorted.drop(['Cluster Labels'], axis = 1, inplace = True)    
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood').drop("City",1), on='Neighborhood')
merged


,City,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Toronto,The Beaches,43.676357,-79.293031,5,Health Food Store,Pub,Astrologer,Zoo,Exhibit,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant
1,Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Caribbean Restaurant,Diner,Liquor Store,Bubble Tea Shop,Bakery
2,Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,5,Park,Sandwich Place,Movie Theater,Gym,Pizza Place,Pet Store,Burrito Place,Burger Joint,Fast Food Restaurant,Fish & Chips Shop
3,Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Italian Restaurant,Bakery,Gastropub,American Restaurant,Stationery Store,Furniture / Home Store,Bookstore,Middle Eastern Restaurant,Brewery
4,Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Falafel Restaurant,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant
5,Toronto,Davisville North,43.712751,-79.390197,5,Food & Drink Shop,Hotel,Gym,Park,Dog Run,Clothing Store,Sandwich Place,Dance Studio,Grocery Store,Breakfast Spot
6,Toronto,North Toronto West,43.715383,-79.405678,5,Clothing Store,Coffee Shop,Gift Shop,Fast Food Restaurant,Salon / Barbershop,Mexican Restaurant,Diner,Chinese Restaurant,Gym,Sporting Goods Shop
7,Toronto,Davisville,43.704324,-79.388790,1,Coffee Shop,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Sushi Restaurant,Gourmet Shop,Pharmacy,Seafood Restaurant,Park
8,Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,3,Summer Camp,Playground,Zoo,Farmers Market,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
9,Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,1,Pub,Coffee Shop,Sports Bar,American Restaurant,Liquor Store,Light Rail Station,Supermarket,Vietnamese Restaurant,Sushi Restaurant,Bagel Shop


In [77]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)
# map_clusters = folium.Map(location=[NY_latitude, NY_longitude], zoom_start=11)
# map_clusters = folium.Map(location=[pr_latitude, pr_longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [76]:
merged.sort_values(by = ["City","Cluster Labels"])

,City,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Coffee Shop,Pizza Place,Mexican Restaurant,Deli / Bodega,Yoga Studio,Sushi Restaurant,Bakery,Caribbean Restaurant,School,Sandwich Place
58,Manhattan,Lower East Side,40.717807,-73.980890,1,Coffee Shop,Ramen Restaurant,Art Gallery,Bakery,Japanese Restaurant,Pizza Place,Cocktail Bar,Sandwich Place,Chinese Restaurant,Restaurant
64,Manhattan,Morningside Heights,40.808000,-73.963896,1,Coffee Shop,Bookstore,American Restaurant,Park,Tennis Court,Deli / Bodega,Food Truck,Burger Joint,Outdoor Sculpture,Frozen Yogurt Shop
77,Manhattan,Hudson Yards,40.756658,-74.000111,1,Coffee Shop,American Restaurant,Gym,Italian Restaurant,Hotel,Ice Cream Shop,Salad Place,Dog Run,Restaurant,Theater
38,Manhattan,Marble Hill,40.876551,-73.910660,5,Sandwich Place,Discount Store,Coffee Shop,Steakhouse,Supplement Shop,Gym,Clothing Store,Department Store,Deli / Bodega,Tennis Stadium
39,Manhattan,Chinatown,40.715618,-73.994279,5,Chinese Restaurant,Cocktail Bar,American Restaurant,Bubble Tea Shop,Spa,Dumpling Restaurant,Vietnamese Restaurant,Bakery,Salon / Barbershop,Sandwich Place
40,Manhattan,Washington Heights,40.851903,-73.936900,5,Coffee Shop,Bakery,Mobile Phone Shop,Grocery Store,Deli / Bodega,Gym,Intersection,Supermarket,Supplement Shop,Sandwich Place
41,Manhattan,Inwood,40.867684,-73.921210,5,Mexican Restaurant,Coffee Shop,Lounge,Pizza Place,Bakery,Frozen Yogurt Shop,Chinese Restaurant,Pharmacy,Park,Deli / Bodega
43,Manhattan,Manhattanville,40.816934,-73.957385,5,Coffee Shop,Park,Chinese Restaurant,Seafood Restaurant,Italian Restaurant,Mexican Restaurant,Indian Restaurant,Fried Chicken Joint,Bakery,Bank
44,Manhattan,Central Harlem,40.815976,-73.943211,5,Chinese Restaurant,African Restaurant,Cosmetics Shop,Gym,American Restaurant,French Restaurant,Seafood Restaurant,Fried Chicken Joint,Public Art,Bookstore


In [161]:
df = grouped
try:
    df.insert(0, 'Cluster Labels', kmeans.labels_)
except:
    df.drop(['Cluster Labels'], axis = 1, inplace = True)
    df.insert(0, 'Cluster Labels', kmeans.labels_)
df    
df = df.groupby(by = "Cluster Labels").mean()
df

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Cluster']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cluster_sorted = pd.DataFrame(columns=columns)
cluster_sorted['Cluster'] = df.index

for ind in np.arange(df.shape[0]):
    cluster_sorted.iloc[ind, 1:] = return_most_common_venues(df.iloc[ind, :], num_top_venues)

cluster_sorted.insert(1, "N. of neighborhoods",neighborhoods_venues_sorted.groupby(by = "Cluster Labels")["Neighborhood"].nunique()) 
cluster_sorted.set_index("Cluster", inplace=True)
cluster_sorted

,N. of neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster,,,,,,,,,,,
0,17,French Restaurant,Hotel,Coffee Shop,Italian Restaurant,Plaza,Bar,Bakery,Bistro,Japanese Restaurant,Restaurant
1,31,Coffee Shop,Italian Restaurant,Park,Pizza Place,Restaurant,Gym,Bakery,Sandwich Place,American Restaurant,Pub
2,1,Garden,Home Service,Zoo,Farmers Market,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
3,4,Park,Trail,Bus Line,Swim School,Jewelry Store,Playground,Sushi Restaurant,Zoo,Pool,Supermarket
4,44,Coffee Shop,Gym,Italian Restaurant,Park,Bakery,Bar,Pizza Place,Hotel,Mexican Restaurant,American Restaurant
5,1,Summer Camp,Playground,Zoo,Farmers Market,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant
